In [5]:
import socket
from IPython.display import clear_output


In [6]:
#!/usr/bin/env python3
#coding=utf-8
import time
import actions as Arm_Actions

from threading import Thread

In [7]:
time_1 = 500
time_2 = 1000
time_sleep = 0.5

recent_model = 2
slider_angles = [90,90,90,90,90,90]
#lock = Thread.Lock()
isRunning = False
detection_index = -1

In [8]:
# Read the angles of all steering gears and print them out circularly
# 读取所有舵机的角度，并打印出来
def read_all():
    for i in range(6):
        aa = Arm_Actions.Arm.Arm_serial_servo_read(i+1)
        print(aa)
        time.sleep(.01)
    time.sleep(.5)
    print(" END OF LINE! ")
    
def enable_all(onoff):
    for i in range(6):
        Arm_Actions.Arm.Arm_serial_set_torque(i+1)
        time.sleep(.5)

def resetSituation():
    Arm_Actions.Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, time_1)
    time.sleep(1)
    

In [9]:

# 'bow','down','eight','five','good','handheart','OK','pray','Rock','seven
def task():
    global recent_model,slider_angles,isRunning,detection_index
    
    try:
        while True:
            #clear_output(wait=True) 
            print(f"I AM runnig: {detection_index} + {recent_model} + {isRunning}")
            if isRunning:
                time.sleep(0.1)
                continue
            elif recent_model == 0:
                print(f"i am control")
                Arm_Actions.Arm.Arm_serial_servo_write6(slider_angles[0], slider_angles[1], slider_angles[2],
                                                        slider_angles[3], slider_angles[4], slider_angles[5], time_1)
                time.sleep(1)
            elif recent_model == 1:
                if 0<=detection_index<=9 :
                    print(f"i am detect: {detection_index}")
                    Arm_Actions.action(mode=recent_model,action=detection_index,angles=slider_angles)
            time.sleep(0.1)
    except Exception as e:
            print(f"Error in task: {e}")

In [10]:
resetSituation()

In [11]:
read_all()

90
91
91
90
89
92
 END OF LINE! 


In [7]:
Arm_Actions.clap()

In [8]:
print(isRunning)
Arm_Actions.action(mode=1,action=2,angles=slider_angles)
print(isRunning)

False
True
False


In [12]:
def start_receiver(host='0.0.0.0', port=12345):
    global recent_model,slider_angles,isRunning,detection_index
    # 创建 TCP 套接字
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))  # 连接到目标主机和端口
    print(f"Connected to server at {host}:{port}")
    thread = Thread(target=task)
    thread.daemon = True # 守护线程
    
    try:
        thread.start()
        print("启动线程")

        while True:
            # 循环接收数据
            data = client_socket.recv(1024)  # 假设最大接收数据为 1024 字节
            if not data:
                print("Connection closed by server.")
                detection = -1
                break

            # 解析接收到的数据
            if len(data) >= 8:  # 至少要有 8 个字节 (1 + 1 + 6)
                recent_model = data[0]
                detection = data[1]
                slider_angles = data[2:8]  # 获取后 6 个字节，对应滑块的角度值
                
                clear_output(wait=True) 
                # 实现指令
                if(not isRunning):
                    detection_index = detection
                print(f"Detected: {detection_index} {isRunning}")

                # 打印解析结果
                print(f"Received data: {data.hex()}")
                print(f"Recent Model: {recent_model}")
                print(f"Detection: {detection}")
                for i, angle in enumerate(slider_angles):
                    print(f"Slider {i+1} angle: {angle} degrees")
            else:
                print("Received data is too short to process.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # 关闭连接
        client_socket.close()

In [ ]:
start_receiver('192.168.1.169',8189)

Detected: 231 False
Received data: 01e75a5a5a5a6581
Recent Model: 1
Detection: 231
Slider 1 angle: 90 degrees
Slider 2 angle: 90 degrees
Slider 3 angle: 90 degrees
Slider 4 angle: 90 degrees
Slider 5 angle: 101 degrees
Slider 6 angle: 129 degrees
I AM runnig: 231 + 1 + False
